In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
import random
from itertools import chain
from collections import Counter
import numpy as np
from scipy.stats import norm
import math

from ipynb.fs.full.GenerateData import generatedata


**0) Define variables**

$ \{x_{1}, x_{2},...x_{k}\} \in X$, Observed features of defendant
 
$\mu_{prior}$, Judge's mean prior belief about a defendant; \textit{most probable} risk prediction

$\sigma_{prior}$, Standard deviation of judge's prior belief on a defendant; corresponds to a judge's $\textit{uncertainty}$ about prediction

$\mu_{ra}$, Observed algorithmic risk assessment prediction; mean of normally-distributed perceived anchor information

$\sigma_{ra}$, Perceived anchor $\textit{influence}$; Corresponds to S.D. of risk assessment (when perceived as containing a lot of information S.D. will be lower and vice versa).

$w,c$, Parameters for mapping $\sigma_{prior}$ onto $\sigma_{ra}$

$z_{\alpha}$, $z$-score distance between $\mu_{prior}$ and $\mu_{ra}$

$z$, Threshold distance between $\mu_{prior}$ and $\mu_{ra}$ after which $\mu_{ra}$ has no (or minimal) effect on $\mu_{prior}$

$\mu_{post}$, Judge's mean posterior belief

$\sigma_{post}$, Standard deviation of judge's posterior belief 

$\tau$, Decision making threshold

$\phi_{prior}(\mu_{prior},\sigma_{prior})$, Gaussian distribution representing judge's prior belief

$\phi_{ra}(\mu_{ra},\sigma_{ra})$, Gaussian distribution representing the influence of algorithmic risk assessment predictions

$\phi_{post}(\mu_{post},\sigma_{post})$, Gaussian distribution representing the judge's posterior belief

$\Phi_{post}$, Probability of drawing a belief from the posterior greater than threshold $\tau$

$y$, Judge's observed decision on defendant
 
$\alpha$, Learning rate


**1) Helper functions**

**(a) Initialize parameters**

$\beta_{1}$ (nx1), $\beta_{2}$ (nx1), $var_{prior} = \sigma_{prior}^2$ (1x1), and $w$ (1x1) are randomly initialized (>0);
$b$ (1x1) and $c$ (1x1) are initialized as zero

In [2]:
def initialize_parameters(X,lower,upper):
    """
    Argument:
    
    Returns:

    """
    # of parameters
    n = X.shape[1]
    
    B = 0.1*np.random.rand(n,1)[0] # randomly initialize the B coefficient
    b = np.random.rand(1)*10#0 # intialize  constants bias, b @ 0
    q = np.random.rand(1)
    var = np.random.rand(1)*10
    tau = random.randint(lower, upper)
    
    parameters = {"B": B,
                  "b": b,
                  "q" : q,
                  "var_prior" : var,
                  "tau" : tau}
    
    return parameters

In [3]:
# create dictionary to store derivatives
def initialize_dicts():
    derivatives = {
        "dL": [],
        "dphipost_dmupost": [],
        "dphipost_dvarpost": [],
        "dphipost_dtau": [],
        "dmupost_dmuprior": [],
        "dvarpost_dq": [],
        "dmupost_dq": [],
        "dvarpost_dvarprior": []
    }

    grads = {
        "dB":[],
        "db":[],
        "dvarprior":[],
        "dq":[],
        "dtau":[]
    }
    
    return derivatives,grads

**2) Forward**

**(a) Estimate mean of prior belief distribution**

Given {$\beta, b, \sigma, \theta, \tau, w$}, want to calculate loss, $\mathcal{L}(\hat{y},y_i)$ using the following steps:

1. Calculate prior mean, $\mu_{prior}$
\begin{equation}
\mu_{prior} = \beta X + b
\end{equation}

Initialize the first estimated prior mean and store in the cache. 
Future calculations will use the function calc_prior_mean

In [4]:
def calc_prior_mean(X,parameters):
    """
    Arguments:
    
    Returns:
    """
    
    B = parameters['B']
    b = parameters['b']
    
    # Calculate judge's prior odds  from defendant features
    #mu_prior = np.dot(B.T,X.T)+b
    mu_prior = (B*X)+b # ALT
    
    return mu_prior

**(b) Estimate the standard deviation of the risk assessment score (i.e. perceived confidence in the anchor info):**

**We're not currently including this in further calculations**

\begin{equation}
\sigma_{ra}^2 = var_{ra} = \left \{
	\begin{array}{ll}
        q \cdot var_{prior} & if |\mu_{ra} - \mu_{prior}|<\Theta \\
        \infty & otherwise
    \end{array}
    \right.
\end{equation}

In [5]:
def calc_var_ra(var_prior,mu_prior,mu_ra,q,theta): 
    """
    Arguments:

    Returns:
    """

    var_ra = [q*var_prior if np.abs(mu_ra[i]-mu_prior[i])>theta else float("inf") for i in range(len(mu_prior))]

    return var_ra 


**(c) Calculate the posterior distribution of a judge's beliefs on defendant, $\phi_{post}(\mu_{post},\sigma_{post}^2)$, and probability of making decision, $\hat{y}$:**



\begin{equation}
\phi_{prior}(\mu_{prior},\sigma_{prior}^2) = \mathcal{N}(\mu_{prior},\sigma_{prior}^2)
\end{equation}


\begin{equation}
    \phi_{ra}(\mu_{ra},\sigma_{ra}^2) = \mathcal{N}(\mu_{ra},\sigma_{ra}^2)
\end{equation}

\begin{equation}
\phi_{post}(\mu_{post},\sigma_{post}^2) =  \mathcal{N}(\mu_{post},\sigma_{post}^2)
\end{equation}

\begin{equation}
    = \frac{\phi_{prior}(\mu_{prior},\sigma_{prior}^2)\phi_{ra}(\mu_{ra},\sigma_{ra}^2)}{\int \phi_{prior}(\mu_{prior},\sigma_{prior}^2)\phi_{ra}(\mu_{ra},\sigma_{ra}^2)}
\end{equation}

where:

\begin{equation}
\mu_{post} = \left \{
	\begin{array}{ll}
        \mu_{prior} \cdot \frac{q}{q+1} + \mu_{ra} \cdot \frac{1}{q+1} & if |\mu_{ra} - \mu_{prior}|<\Theta \\
        \mu_{prior} & otherwise
    \end{array}
    \right.
\end{equation}

\begin{equation}
\sigma_{post}^2 = var_{post} = \left \{
	\begin{array}{ll}
        var_{prior} \cdot \frac{q}{q+1} & if |\mu_{ra} - \mu_{prior}|<\Theta \\
        var_{prior} & otherwise
    \end{array}
    \right.
\end{equation}

In [6]:
def calc_post_mean(mu_prior, mu_ra, q, theta):
    """
    Arguments:

    Returns:
    """

    mu_post = np.array([(mu_prior[i]*(q/(q+1)))+(mu_ra[i]/(q+1)) if np.abs(mu_ra[i]-mu_prior[i])>theta else mu_prior[i] for i in range(len(mu_prior))]).astype(float)

    return mu_post

In [7]:
def calc_post_var(mu_prior, mu_ra, var_prior, q, theta):
    """
    Arguments:
    
    Returns:
    """
    
    var_post = np.array([i[0] for i in [var_prior*(q/(q+1)) if np.abs(mu_ra[i]-mu_prior[i])>theta else var_prior for i in range(len(mu_prior))]]).astype(float)
    
    return var_post

**(d) Calculate the probability of detaining defendant as area under the posterior belief curve $\geq$ decision threshold, $\tau$**

\begin{equation}
\Phi(\tau;\mu_{post},\sigma_{post}) = \int_\tau^{\infty} \phi_{post}(\tau; \mu_{post},\sigma_{post}^2)
\end{equation}


In [8]:
def calc_Phi(mu_post,var_post,tau):    
    """
    Arguments:
    
    Returns:
    """

    Phi = [1 - norm.cdf(tau,loc=mu_post[i],scale=np.sqrt(var_post[i])) for i in range(len(mu_post))]
    
    Phi = [1e-16 if i==0 else i for i in Phi]
    Phi = [1-1e-16 if i==1 else i for i in Phi]

    return np.array(Phi).astype(float)

**(e) Compute negative log likelihood**


\begin{equation}
\mathcal{L}(\Phi(\tau;\mu_{post},\sigma_{post}^2), y_i) = - (y_i \log (1-\Phi(\tau;\mu_{post},\sigma_{post}^2)) + (1-y_i) \log \Phi(\tau;\mu_{post},\sigma_{post}^2))
\end{equation}


In [9]:
def calc_L(Phi,y):
    """
    Arguments:

    Returns:
    """
    
    L = -((y*np.log(1-Phi))+((1-y)*np.log(Phi))) #negative ll
    #L = ((np.array(y)*np.log(1-np.array(Phi)))+(1-np.array(y))*np.log(np.array(Phi))) # log likelihood
    
    return L

**4) Calculate derivatives for gradient descent**

\begin{equation}
    d\mathcal{L} = \phi_{post}\left(\frac{y}{1-\Phi}-\frac{1-y}{\Phi}\right)
\end{equation}

\begin{equation}
    \frac{d\phi_{post}}{d\mu_{post}} = \frac{\tau-\mu_{post}}{\sqrt{2 \pi \cdot var_{post}^{3}}}\cdot exp \left(- \frac{(\tau-\mu_{post})^2}{2\cdot var_{post}} \right)
\end{equation}

\begin{equation}
    \frac{d\phi_{post}}{dvar_{post}} = \left(\frac{(\tau-\mu_{post})^2}{\sqrt{8 \cdot \pi \cdot var_{post}^{5}}} - \frac{1}{\sqrt{8 \cdot \pi \cdot var_{post}^{3}}} \right) \cdot exp \left[- \frac{(\tau-\mu_{post})^2}{2\cdot var_{post}} \right]
\end{equation}

\begin{equation}
    \frac{d\phi_{post}}{d\tau}= -\frac{\tau-\mu_{post}}{\sqrt{2\cdot \pi \cdot var_{post}^{3}}}exp\left[-\frac{(\tau-\mu_{post})^2}{2var_{post}} \right]
\end{equation}

\begin{equation}
    \frac{d\mu_{post}}{d\mu_{prior}} = 
    \left \{
	\begin{array}{ll}
        \frac{q}{q+1} & if |\mu_{ra} - \mu_{prior}|<\Theta \\
        1 & otherwise
    \end{array}
    \right.
\end{equation}

\begin{equation}
    \frac{d\mu_{post}}{dq} = 
        \left \{
	\begin{array}{ll}
        \frac{\mu_{prior}}{1+q} - \frac{\mu_{ra}}{(1+q)^2}-\frac{q\cdot\mu_{prior}}{(1+q)^2} & if |\mu_{ra} - \mu_{prior}|<\Theta \\
        0 & otherwise
    \end{array}
    \right.
\end{equation}        

\begin{equation}
    \frac{dvar_{post}}{dq} = 
    \left \{
	\begin{array}{ll}
        \frac{var_{prior}}{q+1} - \frac{q \cdot var_{prior}}{(q+1)^2} & if |\mu_{ra} - \mu_{prior}|<\Theta \\
        0 & otherwise
    \end{array}
    \right.
\end{equation}

\begin{equation}
    \frac{dvar_{post}}{dvar_{prior}} = 
        \left \{
	\begin{array}{ll}
        \frac{q}{q+1} & if |\mu_{ra} - \mu_{prior}|<\Theta \\
        1 & otherwise
    \end{array}
    \right.
\end{equation}

In [10]:
def calc_component_derivs(X, parameters, derivatives, theta, mu_prior,mu_ra,mu_post,var_post,Phi,y):
    
    #retrieve variables
    B = parameters['B']
    b = parameters['b']
    q = parameters['q']
    var_prior = parameters['var_prior']
    tau = parameters['tau']
    
    # calculate derivatives / partial derivatives
    
    # dL/dphi
    phi_post = (1/np.sqrt(2*math.pi*var_post))*np.exp(-(((tau-mu_post)**2)/(2*var_post)).astype(float))
    dL = np.array(phi_post*((y/(1-Phi))-((1-y)/Phi)))
    
    # exponent - to reuse in following expressions
    exponent = -((tau-mu_post.astype(float))**2)/(2*var_post)
    
    # dphipost_dmupost
    dphipost_dmupost = (tau-mu_post.astype(float))/(np.sqrt(2*math.pi*(var_post**3)))*np.exp(exponent)
    
    # dphipost_dvarpost
    f1 = ((tau-mu_post.astype(float))**2)/np.sqrt(8*math.pi*var_post**(5)) # Alternative
    f2 = 1/np.sqrt(2*math.pi*var_post**(3))
    dphipost_dvarpost = (f1-f2)*np.exp(exponent)
    
    # dphipost_dtau
    dphipost_dtau = -((tau-mu_post.astype(float))/np.sqrt(2*math.pi*(var_post**3)))*np.exp(exponent)
    
    # dmupost_dmuprior
    dmupost_dmuprior = np.array([(q/(q+1)) if np.abs(mu_prior[i] - mu_ra[i])>theta else 1 for i in range(len(mu_prior))]).astype(float)
    
    # dvarpost_dq
    dvarpost_dq = np.array([((var_prior/(q+1))-((q*var_prior)/((q+1)**2)))[0] if np.abs(mu_prior[i] - mu_ra[i])>theta else 0 for i in range(len(mu_prior))]).astype(float)
    
    # dmupost_dq
    dmupost_dq = np.array([((mu_prior[i]/(q+1)) - (mu_ra[i]/((q+1)**2)) - ((mu_prior[i]*q)/((q+1)**2))) if np.abs(mu_prior[i] - mu_ra[i])>theta else 0 for i in range(len(mu_prior))]).astype(float)
    
    # dvarpost_dvarprior
    dvarpost_dvarprior = np.array([(q/(q+1)) if np.abs(mu_prior[i] - mu_ra[i])>theta else 1 for i in range(len(mu_prior))]).astype(float)
    
    derivatives = {
        "dL": dL,
        "dphipost_dmupost": dphipost_dmupost,
        "dphipost_dvarpost": dphipost_dvarpost,
        "dphipost_dtau": dphipost_dtau,
        "dmupost_dmuprior": dmupost_dmuprior,
        "dvarpost_dq": dvarpost_dq,
        "dmupost_dq": dmupost_dq,
        "dvarpost_dvarprior": dvarpost_dvarprior
    }
    
    return derivatives

**5) Use chain rule to calculate gradients**

\begin{equation}
    \frac{d\mathcal{L}}{d\phi} = (1-y)\cdot\frac{\phi_{post}}{\Phi} - y\cdot\frac{\phi_{post}}{1-\Phi} 
\end{equation}

\begin{equation}
    \frac{d\phi_{post}}{d\beta} =  \frac{\partial\phi_{post}}{\partial\mu_{post}}\cdot \frac{\partial\mu_{post}}{\partial\mu_{prior}}\cdot \frac{d\mu_{prior}}{d\beta}
\end{equation}

\begin{equation}
    \frac{d\phi_{post}}{db} =  \frac{\partial\phi_{post}}{\partial\mu_{post}}\cdot \frac{\partial\mu_{post}}{\partial\mu_{prior}}\cdot \frac{d\mu_{prior}}{db}
\end{equation}

\begin{equation}
    \frac{d\phi_{post}}{dvar_{prior}} = \frac{d\phi_{post}}{dvar_{post}} \cdot \frac{dvar_{post}}{dvar_{prior}} 
\end{equation}

\begin{equation}
    \frac{d\phi_{post}}{dq} = \frac{d\phi_{post}}{dvar_{post}} \cdot \frac{dvar_{post}}{dq} + \frac{d\phi_{post}}{d\mu_{post}} \cdot \frac{d\mu_{post}}{dq} 
\end{equation}

\begin{equation}
    \frac{d\phi_{post}}{d\tau}
\end{equation}

In [11]:
def calc_gradients(grads,derivatives,X): 
    # Retrieve variables
    dL = derivatives['dL']
    dphipost_dmupost = derivatives['dphipost_dmupost']
    dphipost_dvarpost = derivatives['dphipost_dvarpost']
    dphipost_dtau = derivatives['dphipost_dtau']
    dmupost_dmuprior = derivatives['dmupost_dmuprior']
    dvarpost_dq = derivatives['dvarpost_dq']
    dmupost_dq = derivatives['dmupost_dq']
    dvarpost_dvarprior = derivatives['dvarpost_dvarprior']

    # dL_dB
    #dL_dB = np.dot(dL*dphipost_dmupost*dmupost_dmuprior,X)
    dL_dB = np.sum(dL*dphipost_dmupost*dmupost_dmuprior*np.array(X.T))
    
    # dL_db
    dL_db = np.sum(dL*dphipost_dmupost*dmupost_dmuprior)

    #dL_dvarprior
    dL_dvarprior = np.sum(dL*dphipost_dvarpost*dvarpost_dvarprior)         

    # dL_dq
    dL_dq = np.sum((dL*dphipost_dvarpost*dvarpost_dq)+(dL*dphipost_dmupost*dmupost_dq))

    # dL_dtau
    dL_dtau = np.sum(dL*dphipost_dtau)

    grads = {
            "dB":dL_dB,
            "db":dL_db,
            "dvarprior":dL_dvarprior,#dvar_dprior
            "dq":dL_dq,
            "dtau":dL_dtau
    }
    
    return grads

**8) Update parameters with gradients**

\begin{equation}
\beta' = \beta - \alpha \frac{d\mathcal{L}}{d\beta} \\
b' = b - \alpha \frac{d\mathcal{L}}{db} \\
q' = q - \alpha \frac{d\mathcal{L}}{dq} \\
var_{prior}' = var_{prior}-\alpha\frac{d\mathcal{L}}{dvar_{prior}} \\
\tau' = \tau - \alpha \frac{dL}{d\tau}
\end{equation}

In [12]:
def update_parameters(parameters, grads, learning_rate):
    """
    Arguments:
    parameters -- dictionary containing parameters 
    grads -- dictionary containing gradients 
    learning rate
    
    Returns:
    parameters -- dictionary containing updated parameters 
    """
    B = parameters['B']
    b = parameters['b']
    q = parameters['q']
    var_prior = parameters['var_prior']
    tau = parameters['tau']
    
    dB = grads['dB'] #dB.shape = (dB.shape[0],1)    
    db = grads['db']
    dq = grads['dq']
    dvarprior = grads['dvarprior']
    dtau = grads['dtau']
    
    B -= learning_rate*dB
    b -= learning_rate*db
    q -= learning_rate*dq
    var_prior -= learning_rate*dvarprior
    tau -= learning_rate*dtau
    
    parameters = {"B": B,
                  "b": b,
                  "q" : q,
                  "var_prior" : var_prior,
                  "tau" : tau}
    
    return parameters

In [13]:
            
"""
    dmupost_dmuprior = []
    dvarpost_dq = []
    dmupost_dq = []
    dvarpost_dvarprior = []
    
    for i in range(len(mu_prior)):
        if np.abs(mu_prior[i] - mu_ra[i]) > theta:
            dmupost_dmuprior.append((q/(q+1))[0]) # dmu_post/dmu_prior
            dvarpost_dq.append(((var_prior/(q+1))-((q*var_prior)/((q+1)**2)))[0]) #dvarpost_dq
            dmupost_dq.append(((mu_prior[i]/(q+1)) - (mu_ra[i]/((q+1)**2)) - ((mu_prior[i]*q)/((q+1)**2)))[0])#dmupost_dq
            dvarpost_dvarprior.append(0) # dvarpost_dvarprior
        else:
            dmupost_dmuprior.append(1) # dmu_post/dmu_prior
            dvarpost_dq.append(0)#dvarpost_dq
            dmupost_dq.append(0)#dmupost_dq
            dvarpost_dvarprior.append(1)#dvarpost_dvarprior    
    
    derivatives['dmupost_dmuprior'] = np.array(dmupost_dmuprior)
    derivatives['dmupost_dq'] = np.array(dmupost_dq)
    derivatives['dvarpost_dvarprior'] = np.array(dvarpost_dvarprior)
    derivatives['dvarpost_dq'] = np.array(dvarpost_dq)
    """

"\n    dmupost_dmuprior = []\n    dvarpost_dq = []\n    dmupost_dq = []\n    dvarpost_dvarprior = []\n    \n    for i in range(len(mu_prior)):\n        if np.abs(mu_prior[i] - mu_ra[i]) > theta:\n            dmupost_dmuprior.append((q/(q+1))[0]) # dmu_post/dmu_prior\n            dvarpost_dq.append(((var_prior/(q+1))-((q*var_prior)/((q+1)**2)))[0]) #dvarpost_dq\n            dmupost_dq.append(((mu_prior[i]/(q+1)) - (mu_ra[i]/((q+1)**2)) - ((mu_prior[i]*q)/((q+1)**2)))[0])#dmupost_dq\n            dvarpost_dvarprior.append(0) # dvarpost_dvarprior\n        else:\n            dmupost_dmuprior.append(1) # dmu_post/dmu_prior\n            dvarpost_dq.append(0)#dvarpost_dq\n            dmupost_dq.append(0)#dmupost_dq\n            dvarpost_dvarprior.append(1)#dvarpost_dvarprior    \n    \n    derivatives['dmupost_dmuprior'] = np.array(dmupost_dmuprior)\n    derivatives['dmupost_dq'] = np.array(dmupost_dq)\n    derivatives['dvarpost_dvarprior'] = np.array(dvarpost_dvarprior)\n    derivatives['dvar

In [ ]:
"""
def calc_component_derivs(X, parameters, derivatives, theta, mu_prior,mu_ra,mu_post,var_post,Phi,y): 
    
    # retrieve variables
    B = parameters['B']
    b = parameters['b']
    q = parameters['q']
    var_prior = parameters['var_prior']
    tau = parameters['tau']

    # Calculate Derivatives / Partial Derivatives

    # dL/dphi
    phi_post = (1/np.sqrt(2*math.pi*np.array(var_post)))*np.exp(-(((tau-np.array(mu_post))**2)/(2*np.array(var_post))))
    derivatives['dL'] = np.array(phi_post*((y/(1-Phi))-((1-y)/Phi)))

    # dphi_post / dmu_post
    #var_post_exp = [vp**(3/2) for vp in var_post]
    #p1 = np.subtract(tau,mu_post)/np.multiply(var_post_exp,((2*math.pi)**(1/2)))
    #p2 = -(np.subtract(tau,mu_post)**2 / np.multiply(2,var_post))
    #derivatives['dphipost_dmupost'] = p1*np.exp(p2.astype(float))
    
    a1 = (tau-mu_post)/(np.sqrt(2*math.pi)*(var_post**(3/2))) # Alternative
    a2 = ((tau-mu_post)**2)/(2*var_post) # alt
    derivatives['dphipost_dmupost'] = a1*np.exp(-a2) # alt
    
    # dphi_post / dvar_post
    #var_post_exp_5 = [vp**(5/2) for vp in var_post]
    #p1 = (np.subtract(tau,mu_post)**2)/np.multiply(np.multiply(2,var_post_exp_5),((2*math.pi)**(1/2)))
    #var_post_exp_3 = [vp**(3/2) for vp in var_post]
    #p2 = 1/(np.multiply(2,var_post_exp_3)*((2*math.pi)**(1/2)))
    #p3 = (-((np.subtract(tau,mu_post)**2)/np.multiply(2,var_post)))
    #derivatives['dphipost_dvarpost'] = (p1-p2)*np.exp(p3.astype(float))
    
    f1 = ((tau-mu_post)**2)/(2*(var_post**(5/2))*np.sqrt(2*math.pi)) # Alternative
    f2 = 1/(2*(var_post**(3/2))*np.sqrt(2*math.pi))
    f3 = ((tau-mu_post)**2)/(2*var_post)
    
    derivatives['dphipost_dvarpost'] = (f1-f2)*np.exp(-f3)
    
    #dphi_post / dtau
    #var_post_exp = [vp**(3/2) for vp in var_post]
    #p1 = np.subtract(tau,mu_post)/np.multiply(var_post_exp,np.sqrt(2*math.pi))
    #p2 = (np.subtract(tau,mu_post)**2)/np.multiply(2,var_post)
    #derivatives['dphipost_dtau'] = -p1*np.exp(-p2)
    
    g1 = -((tau-mu_post)/((np.sqrt(2*math.pi))*(var**(3/2))))
    g2 = ((tau-mu_post)**2)/(2*var)
    derivatives['dphipost_dtau'] = g1*np.exp(-g2)

    # dmu_post/dmu_prior
    #dvarpost_dq
    #dmupost_dq
    #dvarpost_dvarprior
    
    for i in range(len(mu_prior)):
        if np.abs(mu_prior[i] - mu_ra[i]) > theta:
            derivatives['dmupost_dmuprior'].append((q/(q+1))[0])
            derivatives['dvarpost_dq'].append(((var_prior/(q+1))-((q*var_prior)/((q+1)**2)))[0])
            derivatives['dmupost_dq'].append((mu_prior[i]/(q+1)) - (mu_ra[i]/((q+1)**2)) - ((mu_prior[i]*q)/((q+1)**2)))[0])
            derivatives['dvarpost_dvarprior'].append(0)
        else:
            derivatives['dmupost_dmuprior'].append(1)
            derivatives['dvarpost_dq'].append(0)
            derivatives['dmupost_dq'].append(0)
            derivatives['dvarpost_dvarprior'].append(1)
    
    return derivatives
    """